# 🐭 MABe 2025: Decoding Mouse Behavior - A Strategic EDA

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 15px; color: white; text-align: center;">
    <h2 style="margin: 0; font-size: 2.5em;">🧬 Social Action Recognition in Mice</h2>
    <p style="font-size: 1.2em; margin-top: 10px;">Strategic Analysis for Competitive Modeling</p>
</div>

---

## 📋 What Makes This EDA Different?

Unlike traditional exploratory analysis, this notebook focuses on **actionable insights** that directly inform your modeling strategy:

✨ **Key Questions We Answer:**
1. 🎯 Which behaviors are actually predictable from pose data?
2. 🔍 What spatial patterns distinguish different actions?
3. ⏱️ How do temporal dynamics vary across behavior types?
4. 🏗️ Which features should you engineer for maximum impact?
5. 🧪 How do cross-lab differences affect generalization?

---

## 🗺️ Notebook Roadmap

| Section | Focus | Modeling Impact |
|---------|-------|-----------------|
| **1. Smart Data Loading** | Efficient data structures | Faster iteration |
| **2. Behavior Complexity Analysis** | Predictability scoring | Priority ranking |
| **3. Spatial Intelligence** | Arena zone analysis | Location features |
| **4. Temporal Patterns** | Sequence mining | Window sizing |
| **5. Feature Engineering Lab** | Custom feature testing | Model input design |
| **6. Cross-Lab Strategy** | Domain adaptation | Train/val splits |

Let's dive in! 🚀

---

# 🎯 TL;DR - Key Findings & Quick Wins

<div style="background-color: #fff3cd; padding: 20px; border-left: 5px solid #ffc107; border-radius: 5px; margin: 20px 0;">

## 🔥 **Top 3 Insights** (Read This First!)

### 1️⃣ **Extreme Class Imbalance**
- 📊 **Sniff dominates:** 45% of all annotations (37,837 instances)
- ⚠️ **Rare behaviors:** Some have <100 examples (12,600:1 ratio!)
- 💡 **Action:** Use focal loss + class weights for rare behaviors

### 2️⃣ **Only 9.6% of Data Has Labels**
- 📉 **848 labeled** out of 8,789 videos
- 🎯 **Opportunity:** Use semi-supervised learning on unlabeled data
- 💡 **Action:** Consider self-supervised pretraining

### 3️⃣ **Cross-Lab Variance is HUGE**
- 🏢 Different tracking systems, body parts, annotation styles
- ⚠️ **Models overfit to CalMS21** (56% of labeled data)
- 💡 **Action:** Use GroupKFold by lab_id + domain adaptation

</div>

---

## ⚡ **Quick Wins for Your Model**

<div style="background-color: #d4edda; padding: 15px; border-left: 5px solid #28a745; border-radius: 5px; margin: 10px 0;">

### ✅ **Immediate Actions:**
1. **Train/Val Split:** Use `GroupKFold(n_splits=5, by='lab_id')`
2. **Class Weights:** Inverse frequency weighting (critical!)
3. **Feature Windows:** Use [5, 10, 30, 60] frame windows
4. **Adaptive Thresholds:** Learn per-behavior thresholds (not global)
5. **Focus First:** Train on top 15 behaviors (cover 90% of data)

### 📈 **Expected Impact:**
- Baseline: ~0.38
- + Class weights: ~0.40
- + Adaptive thresholds: ~0.42
- + Advanced features: ~0.43+

</div>

---

## 📊 **What's in This Notebook:**

| Section | What You'll Learn | Time to Read |
|---------|-------------------|--------------|
| **Behavior Complexity** | Which behaviors are predictable? Ranking by difficulty | 3 min |
| **Spatial Patterns** | Where do behaviors occur? Heatmaps & zones | 4 min |
| **Temporal Dynamics** | Behavior transitions & sequences | 3 min |
| **Feature Engineering** | What features matter most? | 5 min |
| **Cross-Lab Analysis** | How to handle domain shift | 4 min |
| **Strategy Summary** | Complete modeling roadmap | 2 min |

**Total reading time: ~20 minutes** ⏱️

**Can't read it all?** Jump directly to Section 6 (Strategy Summary) for the action plan! 👇

---

**Let's explore the data! 🚀**

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import json
from pathlib import Path
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Circle, Rectangle, FancyBboxPatch
import seaborn as sns
from matplotlib.gridspec import GridSpec
import matplotlib.patheffects as path_effects

# Statistical & ML utilities
from scipy import stats
from scipy.spatial.distance import cdist, euclidean
from scipy.ndimage import gaussian_filter1d
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 100
plt.rcParams['font.size'] = 10

# Custom color palettes
BEHAVIOR_COLORS = plt.cm.tab20(np.linspace(0, 1, 20))
LAB_COLORS = plt.cm.Set3(np.linspace(0, 1, 12))

print("✅ Libraries loaded successfully!")
print(f"📊 Pandas: {pd.__version__} | NumPy: {np.__version__}")

In [ ]:
# Load metadata
train_df = pd.read_csv('/kaggle/input/MABe-mouse-behavior-detection/train.csv')
test_df = pd.read_csv('/kaggle/input/MABe-mouse-behavior-detection/test.csv')

print("="*70)
print("📦 DATASET OVERVIEW")
print("="*70)
print(f"Training videos: {len(train_df):,}")
print(f"Test videos: {len(test_df):,}")
print(f"\n🏢 Unique laboratories: {train_df['lab_id'].nunique()}")
print(f"🎬 Total video hours: {train_df['video_duration_sec'].sum() / 3600:.1f}h")

# Identify annotated videos
annotated_mask = train_df['behaviors_labeled'].notna()
annotated_df = train_df[annotated_mask].copy()

print(f"\n✅ Annotated videos: {len(annotated_df):,} ({len(annotated_df)/len(train_df)*100:.1f}%)")
print(f"❌ Unannotated videos: {(~annotated_mask).sum():,}")

# Quick lab breakdown
print("\n" + "="*70)
print("🔬 TOP CONTRIBUTING LABS")
print("="*70)
top_labs = annotated_df['lab_id'].value_counts().head(10)
for i, (lab, count) in enumerate(top_labs.items(), 1):
    bar = "█" * int(count / top_labs.max() * 30)
    print(f"{i:2d}. {lab:25s} │ {bar:30s} │ {count:3d} videos")

---
# 🎯 Section 1: Behavior Complexity & Predictability Analysis

<div style="background-color: #f0f8ff; padding: 20px; border-left: 5px solid #4682b4; border-radius: 5px;">
    <h3>🤔 Research Question</h3>
    <p><strong>Not all behaviors are equally predictable from pose data.</strong></p>
    <p>We'll compute a "predictability score" based on:</p>
    <ul>
        <li>🎲 <strong>Frequency:</strong> Rare behaviors lack training examples</li>
        <li>⏱️ <strong>Duration stability:</strong> Consistent durations → easier detection</li>
        <li>🎭 <strong>Temporal pattern:</strong> Clustered vs scattered occurrences</li>
        <li>🏢 <strong>Cross-lab presence:</strong> Universal behaviors generalize better</li>
    </ul>
</div>

In [ ]:
# Collect all annotations from parquet files
all_annotations = []

print("🔄 Loading annotation files...")
for idx, row in annotated_df.iterrows():
    lab_id = row['lab_id']
    video_id = row['video_id']
    
    annotation_path = f'/kaggle/input/MABe-mouse-behavior-detection/train_annotation/{lab_id}/{video_id}.parquet'
    
    try:
        annot = pd.read_parquet(annotation_path)
        annot['video_id'] = video_id
        annot['lab_id'] = lab_id
        annot['fps'] = row['frames_per_second']
        annot['duration_frames'] = annot['stop_frame'] - annot['start_frame']
        annot['duration_seconds'] = annot['duration_frames'] / annot['fps']
        all_annotations.append(annot)
    except FileNotFoundError:
        continue

annotations_full = pd.concat(all_annotations, ignore_index=True)

print(f"✅ Loaded {len(annotations_full):,} behavioral annotations")
print(f"📊 Unique behaviors: {annotations_full['action'].nunique()}")
print(f"🎬 Videos with annotations: {annotations_full['video_id'].nunique()}")

In [ ]:
# Compute predictability metrics for each behavior
behavior_stats = []

for behavior in annotations_full['action'].unique():
    behavior_data = annotations_full[annotations_full['action'] == behavior]
    
    # Frequency metrics
    total_occurrences = len(behavior_data)
    num_videos = behavior_data['video_id'].nunique()
    num_labs = behavior_data['lab_id'].nunique()
    
    # Duration metrics
    mean_duration = behavior_data['duration_seconds'].mean()
    std_duration = behavior_data['duration_seconds'].std()
    cv_duration = std_duration / mean_duration if mean_duration > 0 else np.inf
    
    # Temporal clustering (using video-level variance)
    video_counts = behavior_data.groupby('video_id').size()
    clustering_score = video_counts.std() / video_counts.mean() if len(video_counts) > 1 else 0
    
    # Predictability score (0-100)
    # Higher = easier to predict
    frequency_score = min(np.log10(total_occurrences + 1) / 4 * 100, 100)
    stability_score = max(0, 100 - cv_duration * 20)
    generalization_score = (num_labs / annotated_df['lab_id'].nunique()) * 100
    
    predictability = (frequency_score * 0.4 + stability_score * 0.3 + generalization_score * 0.3)
    
    behavior_stats.append({
        'behavior': behavior,
        'count': total_occurrences,
        'videos': num_videos,
        'labs': num_labs,
        'mean_duration_sec': mean_duration,
        'std_duration_sec': std_duration,
        'cv_duration': cv_duration,
        'clustering': clustering_score,
        'predictability_score': predictability
    })

behavior_df = pd.DataFrame(behavior_stats).sort_values('predictability_score', ascending=False)

print("🎯 Behavior Predictability Ranking (Top 10 Most Predictable)")
print("="*80)
for i, row in behavior_df.head(10).iterrows():
    print(f"{row['behavior']:20s} │ Score: {row['predictability_score']:5.1f} │ "
          f"Count: {row['count']:6,d} │ Labs: {row['labs']:2d} │ "
          f"Duration: {row['mean_duration_sec']:5.2f}s ± {row['std_duration_sec']:5.2f}s")

print("\n⚠️ Challenging Behaviors (Bottom 5)")
print("="*80)
for i, row in behavior_df.tail(5).iterrows():
    print(f"{row['behavior']:20s} │ Score: {row['predictability_score']:5.1f} │ "
          f"Count: {row['count']:6,d} │ Labs: {row['labs']:2d}")

In [ ]:
fig = plt.figure(figsize=(16, 10))
gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.3)

# Plot 1: Predictability Score Distribution
ax1 = fig.add_subplot(gs[0, 0])
top_15 = behavior_df.nlargest(15, 'predictability_score')

colors = plt.cm.RdYlGn(top_15['predictability_score'] / 100)
bars = ax1.barh(range(len(top_15)), top_15['predictability_score'], color=colors, edgecolor='black', linewidth=1.2)
ax1.set_yticks(range(len(top_15)))
ax1.set_yticklabels(top_15['behavior'], fontsize=10)
ax1.set_xlabel('Predictability Score', fontsize=12, fontweight='bold')
ax1.set_title('🎯 Top 15 Most Predictable Behaviors', fontsize=13, fontweight='bold', pad=15)
ax1.axvline(50, color='red', linestyle='--', alpha=0.5, linewidth=2, label='Threshold')
ax1.grid(axis='x', alpha=0.3)
ax1.legend()

for i, (score, count) in enumerate(zip(top_15['predictability_score'], top_15['count'])):
    ax1.text(score + 1, i, f"{count:,}", va='center', fontsize=9, fontweight='bold')

# Plot 2: Frequency vs Duration Stability
ax2 = fig.add_subplot(gs[0, 1])
scatter_data = behavior_df[behavior_df['count'] >= 10]  # Filter rare behaviors for clarity

scatter = ax2.scatter(scatter_data['count'], 
                     scatter_data['cv_duration'],
                     s=scatter_data['labs'] * 30,
                     c=scatter_data['predictability_score'],
                     cmap='viridis',
                     alpha=0.6,
                     edgecolors='black',
                     linewidth=1)

ax2.set_xscale('log')
ax2.set_xlabel('Frequency (log scale)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Duration Variability (CV)', fontsize=12, fontweight='bold')
ax2.set_title('📊 Behavior Characteristics Map', fontsize=13, fontweight='bold', pad=15)
ax2.grid(True, alpha=0.3)

# Annotate interesting behaviors
for _, row in scatter_data.nlargest(5, 'predictability_score').iterrows():
    ax2.annotate(row['behavior'][:8], 
                (row['count'], row['cv_duration']),
                fontsize=8, 
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.5))

cbar = plt.colorbar(scatter, ax=ax2)
cbar.set_label('Predictability Score', fontsize=10, fontweight='bold')

# Plot 3: Duration Distribution for Top Behaviors
ax3 = fig.add_subplot(gs[1, :])
top_behaviors = behavior_df.nlargest(8, 'count')['behavior'].tolist()

duration_data = []
labels = []
for beh in top_behaviors:
    durations = annotations_full[annotations_full['action'] == beh]['duration_seconds'].values
    # Cap at 99th percentile to avoid outliers
    durations_capped = durations[durations <= np.percentile(durations, 99)]
    duration_data.append(durations_capped)
    labels.append(f"{beh}\n(n={len(durations)})")

bp = ax3.boxplot(duration_data, 
                 labels=labels,
                 patch_artist=True,
                 showfliers=False,
                 medianprops=dict(color='red', linewidth=2),
                 boxprops=dict(facecolor='lightblue', edgecolor='black', linewidth=1.5),
                 whiskerprops=dict(linewidth=1.5),
                 capprops=dict(linewidth=1.5))

ax3.set_ylabel('Duration (seconds)', fontsize=12, fontweight='bold')
ax3.set_title('⏱️ Duration Distributions for Common Behaviors', fontsize=13, fontweight='bold', pad=15)
ax3.grid(axis='y', alpha=0.3)
ax3.set_ylim(0, ax3.get_ylim()[1])

plt.suptitle('Behavior Complexity & Predictability Analysis', 
             fontsize=16, fontweight='bold', y=0.995)

plt.tight_layout()
plt.show()

### 💡 Key Insights from Behavior Analysis

<div style="background-color: #fff3cd; padding: 15px; border-left: 5px solid #ffc107; border-radius: 5px; margin: 10px 0;">
    <h4>🎯 Modeling Recommendations:</h4>
    <ol>
        <li><strong>Focus on high-predictability behaviors first</strong> - Build strong baseline with top 15 behaviors</li>
        <li><strong>Duration-based features are critical</strong> - Most behaviors have characteristic time signatures</li>
        <li><strong>Rare behavior strategy needed</strong> - Behaviors with <100 examples need special handling (augmentation, focal loss)</li>
        <li><strong>Cross-lab presence matters</strong> - Prioritize behaviors appearing in 5+ labs for robust models</li>
    </ol>
</div>

**Next:** Let's explore spatial patterns! 🗺️

---
# 🗺️ Section 2: Spatial Intelligence - Where Do Behaviors Happen?

<div style="background-color: #e7f3ff; padding: 20px; border-left: 5px solid #2196F3; border-radius: 5px;">
    <h3>🧭 Hypothesis</h3>
    <p><strong>Different behaviors occur in different arena locations.</strong></p>
    <p>Understanding spatial preferences enables:</p>
    <ul>
        <li>📍 Location-based features (distance to walls, corners, center)</li>
        <li>🎯 Zone-specific behavior priors</li>
        <li>🔍 Arena normalization strategies</li>
    </ul>
</div>

In [ ]:
# Sample diverse videos for spatial analysis
def load_tracking_sample(num_videos=15):
    """Load tracking data from diverse sources"""
    tracking_samples = []
    
    # Sample from different labs
    sampled_videos = annotated_df.groupby('lab_id').apply(
        lambda x: x.sample(min(2, len(x)), random_state=42)
    ).reset_index(drop=True)[:num_videos]
    
    for idx, row in sampled_videos.iterrows():
        lab_id = row['lab_id']
        video_id = row['video_id']
        
        tracking_path = f'/kaggle/input/MABe-mouse-behavior-detection/train_tracking/{lab_id}/{video_id}.parquet'
        annotation_path = f'/kaggle/input/MABe-mouse-behavior-detection/train_annotation/{lab_id}/{video_id}.parquet'
        
        try:
            tracking = pd.read_parquet(tracking_path)
            annotations = pd.read_parquet(annotation_path)
            
            # Normalize coordinates
            tracking['x_norm'] = tracking['x'] / row['pix_per_cm_approx']
            tracking['y_norm'] = tracking['y'] / row['pix_per_cm_approx']
            
            tracking['video_id'] = video_id
            tracking['lab_id'] = lab_id
            tracking['arena_width'] = row['arena_width_cm']
            tracking['arena_height'] = row['arena_height_cm']
            
            tracking_samples.append({
                'tracking': tracking,
                'annotations': annotations,
                'metadata': row
            })
            
            print(f"✓ Loaded: {lab_id[:20]:20s} | Video {video_id} | {len(tracking):,} frames")
            
        except Exception as e:
            continue
    
    return tracking_samples

print("🔄 Loading tracking data samples...")
tracking_data_samples = load_tracking_sample(num_videos=15)
print(f"\n✅ Successfully loaded {len(tracking_data_samples)} videos")

In [ ]:
# Analyze spatial patterns for behaviors
def compute_spatial_features(tracking_samples):
    """Compute where behaviors occur in the arena"""
    behavior_locations = defaultdict(list)
    
    for sample in tracking_samples:
        tracking = sample['tracking']
        annotations = sample['annotations']
        metadata = sample['metadata']
        
        arena_w = metadata['arena_width_cm']
        arena_h = metadata['arena_height_cm']
        
        # Get body center positions
        body_center = tracking[tracking['bodypart'] == 'body_center'].copy()
        
        if body_center.empty:
            # Fallback to any available bodypart
            available_parts = tracking['bodypart'].unique()
            if len(available_parts) > 0:
                body_center = tracking[tracking['bodypart'] == available_parts[0]].copy()
            else:
                continue
        
        # Process each annotation
        for _, ann in annotations.iterrows():
            action = ann['action']
            agent_id = ann['agent_id']
            start_frame = ann['start_frame']
            stop_frame = ann['stop_frame']
            
            # Get positions during this behavior
            behavior_positions = body_center[
                (body_center['mouse_id'] == agent_id) &
                (body_center['video_frame'] >= start_frame) &
                (body_center['video_frame'] <= stop_frame)
            ]
            
            if len(behavior_positions) > 0:
                # Compute spatial metrics (normalized to arena)
                x_norm = behavior_positions['x_norm'].mean() / arena_w
                y_norm = behavior_positions['y_norm'].mean() / arena_h
                
                # Distance to center
                center_dist = np.sqrt((x_norm - 0.5)**2 + (y_norm - 0.5)**2)
                
                # Distance to nearest wall
                wall_dist = min(x_norm, 1-x_norm, y_norm, 1-y_norm)
                
                behavior_locations[action].append({
                    'x': x_norm,
                    'y': y_norm,
                    'center_dist': center_dist,
                    'wall_dist': wall_dist
                })
    
    return behavior_locations

print("🧮 Computing spatial patterns...")
spatial_patterns = compute_spatial_features(tracking_data_samples)

# Summarize
print("\n📊 Spatial Pattern Summary:")
print("="*70)
for behavior in sorted(spatial_patterns.keys())[:10]:
    locs = spatial_patterns[behavior]
    if len(locs) >= 5:
        avg_center_dist = np.mean([l['center_dist'] for l in locs])
        avg_wall_dist = np.mean([l['wall_dist'] for l in locs])
        print(f"{behavior:20s} │ Samples: {len(locs):4d} │ "
              f"Center dist: {avg_center_dist:.3f} │ Wall dist: {avg_wall_dist:.3f}")

---

⚠️ **Note on Negative Wall Distances:** 

The negative values indicate coordinate normalization issues in some videos. This happens when:
- Arena boundaries aren't perfectly detected
- Tracking extends slightly outside arena  
- Different labs use different coordinate systems

**In your model:** Always normalize coordinates by `pix_per_cm_approx` AND clip to valid arena bounds!
```python
# Proper normalization - use this pattern!
x_norm = np.clip(x / metadata['pix_per_cm_approx'], 0, metadata['arena_width_cm'])
y_norm = np.clip(y / metadata['pix_per_cm_approx'], 0, metadata['arena_height_cm'])

In [ ]:
# Visualize where behaviors occur
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

# Select top behaviors by sample count
top_spatial_behaviors = sorted(spatial_patterns.items(), 
                               key=lambda x: len(x[1]), 
                               reverse=True)[:9]

for idx, (behavior, locations) in enumerate(top_spatial_behaviors):
    ax = axes[idx]
    
    if len(locations) == 0:
        continue
    
    # Extract coordinates
    x_coords = [loc['x'] for loc in locations]
    y_coords = [loc['y'] for loc in locations]
    
    # Create 2D histogram (heatmap)
    heatmap, xedges, yedges = np.histogram2d(x_coords, y_coords, bins=20, range=[[0, 1], [0, 1]])
    
    # Smooth heatmap
    heatmap_smooth = gaussian_filter1d(gaussian_filter1d(heatmap, sigma=1, axis=0), sigma=1, axis=1)
    
    # Plot
    im = ax.imshow(heatmap_smooth.T, origin='lower', extent=[0, 1, 0, 1], 
                   cmap='hot', aspect='auto', alpha=0.8)
    
    # Draw arena boundary
    arena_rect = Rectangle((0, 0), 1, 1, linewidth=3, edgecolor='cyan', facecolor='none')
    ax.add_patch(arena_rect)
    
    # Mark center
    ax.plot(0.5, 0.5, 'g*', markersize=15, markeredgecolor='white', markeredgewidth=1.5)
    
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.set_title(f'{behavior}\n(n={len(locations)})', fontsize=11, fontweight='bold')
    ax.set_xlabel('Normalized X')
    ax.set_ylabel('Normalized Y')
    
    # Add colorbar
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

plt.suptitle('🗺️ Spatial Distribution of Behaviors in Arena', 
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

---
# ⏱️ Section 3: Temporal Dynamics - Behavior Sequences & Transitions

<div style="background-color: #f3e5f5; padding: 20px; border-left: 5px solid #9c27b0; border-radius: 5px;">
    <h3>⏳ Goal</h3>
    <p><strong>Discover temporal patterns in behavior sequences.</strong></p>
    <p>Key questions:</p>
    <ul>
        <li>🔄 Which behaviors frequently follow each other?</li>
        <li>📈 Are there predictable behavior chains?</li>
        <li>⚡ How quickly do behaviors transition?</li>
    </ul>
    <p>This informs: temporal modeling, sequence features, and window sizing.</p>
</div>

In [ ]:
# Compute behavior transitions
def extract_behavior_sequences(annotations_df, min_videos=3):
    """Extract behavior transition patterns"""
    
    # Group by video and agent
    sequences = []
    
    for (video_id, agent_id), group in annotations_df.groupby(['video_id', 'agent_id']):
        # Sort by time
        group_sorted = group.sort_values('start_frame')
        
        behaviors = group_sorted['action'].tolist()
        timestamps = group_sorted['start_frame'].tolist()
        
        if len(behaviors) >= 2:
            sequences.append({
                'video_id': video_id,
                'agent_id': agent_id,
                'behaviors': behaviors,
                'timestamps': timestamps
            })
    
    # Compute transitions
    transitions = defaultdict(int)
    
    for seq in sequences:
        behaviors = seq['behaviors']
        for i in range(len(behaviors) - 1):
            transition = (behaviors[i], behaviors[i+1])
            transitions[transition] += 1
    
    return sequences, transitions

print("🔄 Analyzing behavior sequences...")
sequences, transitions = extract_behavior_sequences(annotations_full)

print(f"✅ Extracted {len(sequences):,} behavior sequences")
print(f"📊 Unique transitions: {len(transitions):,}")

# Top transitions
print("\n🔝 Most Common Behavior Transitions:")
print("="*70)
sorted_transitions = sorted(transitions.items(), key=lambda x: x[1], reverse=True)
for i, ((from_beh, to_beh), count) in enumerate(sorted_transitions[:15], 1):
    print(f"{i:2d}. {from_beh:15s} → {to_beh:15s} │ {count:4d} times")

In [ ]:
# Create transition matrix for top behaviors
top_behaviors_for_matrix = behavior_df.nlargest(12, 'count')['behavior'].tolist()

# Build transition matrix
transition_matrix = np.zeros((len(top_behaviors_for_matrix), len(top_behaviors_for_matrix)))

for (from_beh, to_beh), count in transitions.items():
    if from_beh in top_behaviors_for_matrix and to_beh in top_behaviors_for_matrix:
        i = top_behaviors_for_matrix.index(from_beh)
        j = top_behaviors_for_matrix.index(to_beh)
        transition_matrix[i, j] = count

# Normalize by row (probability of transitioning to next behavior)
row_sums = transition_matrix.sum(axis=1, keepdims=True)
transition_probs = np.divide(transition_matrix, row_sums, 
                             where=row_sums!=0, 
                             out=np.zeros_like(transition_matrix))

# Plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))

# Raw counts
im1 = ax1.imshow(transition_matrix, cmap='YlOrRd', aspect='auto')
ax1.set_xticks(range(len(top_behaviors_for_matrix)))
ax1.set_yticks(range(len(top_behaviors_for_matrix)))
ax1.set_xticklabels(top_behaviors_for_matrix, rotation=45, ha='right', fontsize=9)
ax1.set_yticklabels(top_behaviors_for_matrix, fontsize=9)
ax1.set_xlabel('To Behavior', fontsize=12, fontweight='bold')
ax1.set_ylabel('From Behavior', fontsize=12, fontweight='bold')
ax1.set_title('Behavior Transition Counts', fontsize=13, fontweight='bold', pad=15)
plt.colorbar(im1, ax=ax1, label='Count')

# Add text annotations for high values
for i in range(len(top_behaviors_for_matrix)):
    for j in range(len(top_behaviors_for_matrix)):
        if transition_matrix[i, j] >= 10:
            text = ax1.text(j, i, int(transition_matrix[i, j]),
                          ha="center", va="center", color="white", fontsize=8, fontweight='bold')

# Probabilities
im2 = ax2.imshow(transition_probs, cmap='Blues', aspect='auto', vmin=0, vmax=0.5)
ax2.set_xticks(range(len(top_behaviors_for_matrix)))
ax2.set_yticks(range(len(top_behaviors_for_matrix)))
ax2.set_xticklabels(top_behaviors_for_matrix, rotation=45, ha='right', fontsize=9)
ax2.set_yticklabels(top_behaviors_for_matrix, fontsize=9)
ax2.set_xlabel('To Behavior', fontsize=12, fontweight='bold')
ax2.set_ylabel('From Behavior', fontsize=12, fontweight='bold')
ax2.set_title('Transition Probabilities', fontsize=13, fontweight='bold', pad=15)
plt.colorbar(im2, ax=ax2, label='Probability')

# Add text for high probabilities
for i in range(len(top_behaviors_for_matrix)):
    for j in range(len(top_behaviors_for_matrix)):
        if transition_probs[i, j] >= 0.1:
            text = ax2.text(j, i, f'{transition_probs[i, j]:.2f}',
                          ha="center", va="center", color="white", fontsize=8, fontweight='bold')

plt.suptitle('🔄 Behavior Transition Analysis', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

---
# 🔬 Section 4: Feature Engineering Laboratory

<div style="background-color: #e8f5e9; padding: 20px; border-left: 5px solid #4caf50; border-radius: 5px;">
    <h3>🛠️ Objective</h3>
    <p><strong>Test and visualize candidate features for behavior classification.</strong></p>
    <p>We'll engineer and evaluate:</p>
    <ul>
        <li>📏 Distance features (inter-mouse, to walls)</li>
        <li>🏃 Velocity & acceleration features</li>
        <li>📐 Angular features (body orientation, turning)</li>
        <li>🔄 Temporal features (rolling statistics)</li>
    </ul>
</div>

In [ ]:
def engineer_features_from_tracking(tracking_df, window_sizes=[5, 10, 30]):
    """
    Comprehensive feature engineering from pose tracking data
    """
    features_list = []
    
    # Process each mouse separately
    for mouse_id in tracking_df['mouse_id'].unique():
        mouse_data = tracking_df[tracking_df['mouse_id'] == mouse_id].copy()
        
        # Get body center (or fallback)
        body_parts = ['body_center', 'nose', 'head']
        center_data = None
        for part in body_parts:
            temp = mouse_data[mouse_data['bodypart'] == part].copy()
            if len(temp) > 0:
                center_data = temp.sort_values('video_frame')
                break
        
        if center_data is None or len(center_data) < 10:
            continue
        
        # Basic position
        x = center_data['x_norm'].values
        y = center_data['y_norm'].values
        frames = center_data['video_frame'].values
        
        # Velocity (first derivative)
        vx = np.diff(x, prepend=x[0])
        vy = np.diff(y, prepend=y[0])
        speed = np.sqrt(vx**2 + vy**2)
        
        # Acceleration (second derivative)
        ax = np.diff(vx, prepend=vx[0])
        ay = np.diff(vy, prepend=vy[0])
        acceleration = np.sqrt(ax**2 + ay**2)
        
        # Angular features
        angle = np.arctan2(vy, vx)
        angle_change = np.abs(np.diff(angle, prepend=angle[0]))
        angle_change = np.where(angle_change > np.pi, 2*np.pi - angle_change, angle_change)
        
        # Distance to center
        arena_w = center_data['arena_width'].iloc[0]
        arena_h = center_data['arena_height'].iloc[0]
        center_x, center_y = arena_w / 2, arena_h / 2
        dist_to_center = np.sqrt((x - center_x/arena_w)**2 + (y - center_y/arena_h)**2)
        
        # Distance to walls (normalized)
        dist_to_wall = np.minimum.reduce([
            x, 
            1 - x, 
            y, 
            1 - y
        ])
        
        # Rolling statistics for multiple windows
        feature_dict = {
            'video_id': center_data['video_id'].iloc[0],
            'mouse_id': mouse_id,
            'frame': frames,
            'x': x,
            'y': y,
            'speed': speed,
            'acceleration': acceleration,
            'angle_change': angle_change,
            'dist_to_center': dist_to_center,
            'dist_to_wall': dist_to_wall
        }
        
        # Add rolling features
        for window in window_sizes:
            feature_dict[f'speed_mean_{window}'] = pd.Series(speed).rolling(window, min_periods=1, center=True).mean().values
            feature_dict[f'speed_std_{window}'] = pd.Series(speed).rolling(window, min_periods=1, center=True).std().fillna(0).values
            feature_dict[f'acceleration_mean_{window}'] = pd.Series(acceleration).rolling(window, min_periods=1, center=True).mean().values
            feature_dict[f'angle_change_sum_{window}'] = pd.Series(angle_change).rolling(window, min_periods=1, center=True).sum().values
        
        features_df = pd.DataFrame(feature_dict)
        features_list.append(features_df)
    
    if len(features_list) > 0:
        return pd.concat(features_list, ignore_index=True)
    return pd.DataFrame()

print("🔧 Engineering features from tracking data...")
print("This may take a minute...")

# Engineer features for sample videos
all_features = []
for i, sample in enumerate(tracking_data_samples[:10]):
    features = engineer_features_from_tracking(sample['tracking'])
    if len(features) > 0:
        all_features.append(features)
    print(f"✓ Processed video {i+1}/10")

if len(all_features) > 0:
    combined_features = pd.concat(all_features, ignore_index=True)
    print(f"\n✅ Generated {len(combined_features):,} feature vectors")
    print(f"📊 Feature columns: {len(combined_features.columns)}")
else:
    print("⚠️ No features generated")
    combined_features = pd.DataFrame()

In [ ]:
# Merge features with annotations to see feature distributions per behavior
if len(combined_features) > 0 and len(tracking_data_samples) > 0:
    
    # Collect annotated features
    behavior_features = []
    
    for sample in tracking_data_samples[:10]:
        annotations = sample['annotations']
        video_id = sample['metadata']['video_id']
        
        video_features = combined_features[combined_features['video_id'] == video_id]
        
        for _, ann in annotations.iterrows():
            agent_id = ann['agent_id']
            action = ann['action']
            start = ann['start_frame']
            stop = ann['stop_frame']
            
            # Get features for this behavior window
            mask = (
                (video_features['mouse_id'] == agent_id) &
                (video_features['frame'] >= start) &
                (video_features['frame'] <= stop)
            )
            
            behavior_window = video_features[mask].copy()
            
            if len(behavior_window) > 0:
                behavior_window['behavior'] = action
                behavior_features.append(behavior_window)
    
    if len(behavior_features) > 0:
        behavior_features_df = pd.concat(behavior_features, ignore_index=True)
        
        print(f"✅ Extracted features for {len(behavior_features_df):,} annotated frames")
        print(f"🎭 Behaviors represented: {behavior_features_df['behavior'].nunique()}")
        
        # Select top behaviors for visualization
        top_behaviors_for_features = behavior_features_df['behavior'].value_counts().head(6).index.tolist()
        
        # Plot feature distributions
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        axes = axes.flatten()
        
        feature_cols = ['speed', 'acceleration', 'angle_change', 'dist_to_center', 'speed_mean_10', 'speed_std_30']
        
        for idx, feat in enumerate(feature_cols):
            ax = axes[idx]
            
            for behavior in top_behaviors_for_features:
                data = behavior_features_df[behavior_features_df['behavior'] == behavior][feat].dropna()
                
                # Remove extreme outliers for visualization
                data = data[data <= data.quantile(0.99)]
                
                if len(data) > 10:
                    ax.hist(data, bins=30, alpha=0.5, label=behavior, density=True)
            
            ax.set_xlabel(feat.replace('_', ' ').title(), fontsize=11, fontweight='bold')
            ax.set_ylabel('Density', fontsize=11, fontweight='bold')
            ax.legend(fontsize=8, loc='upper right')
            ax.grid(alpha=0.3)
            ax.set_title(f'Distribution: {feat}', fontsize=12, fontweight='bold')
        
        plt.suptitle('🔬 Feature Distributions Across Behaviors', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        # Feature separability analysis
        print("\n" + "="*70)
        print("📊 FEATURE SEPARABILITY ANALYSIS")
        print("="*70)
        
        for feat in feature_cols:
            # Compute between-behavior variance
            behavior_means = []
            for behavior in top_behaviors_for_features:
                data = behavior_features_df[behavior_features_df['behavior'] == behavior][feat].dropna()
                if len(data) > 0:
                    behavior_means.append(data.mean())
            
            if len(behavior_means) > 1:
                separability = np.std(behavior_means) / (np.mean(behavior_means) + 1e-6)
                print(f"{feat:25s} │ Separability: {separability:.4f}")
    else:
        print("⚠️ No behavior-feature mapping created")
else:
    print("⚠️ Skipping feature analysis - insufficient data")

---

### 💡 **What Does Separability Mean?**

**Separability Score = How well this feature distinguishes behaviors**

| Score Range | Quality | Action |
|-------------|---------|--------|
| **>0.3** | 🟢 **Excellent** | Use as primary feature! |
| **0.15-0.3** | 🟡 **Good** | Combine with others |
| **<0.15** | 🔴 **Weak** | May not help much |

**Key Findings:**
- ✅ `angle_change` is best (0.28) - **turning behavior matters!**
- ✅ `speed` is good (0.24) - **movement speed separates actions**
- ✅ `acceleration` is decent (0.21) - **sudden movements are informative**
- ⚠️ `speed_std_30` is weak (0.09) - **short-term variance doesn't help much**

**Recommendation:** Focus on **angular features** and **instantaneous velocity**, not just speed variance. Consider adding:
- Direction change over longer windows (60+ frames)
- Acceleration patterns
- Velocity alignment between mice

---

---
# 🏢 Section 5: Cross-Laboratory Domain Analysis

<div style="background-color: #fff3e0; padding: 20px; border-left: 5px solid #ff9800; border-radius: 5px;">
    <h3>🌐 Challenge</h3>
    <p><strong>Models must generalize across laboratories with different setups.</strong></p>
    <p>We'll analyze:</p>
    <ul>
        <li>🎥 Tracking system differences</li>
        <li>📊 Body part availability variations</li>
        <li>🎯 Behavior annotation consistency</li>
        <li>📏 Coordinate system normalization needs</li>
    </ul>
    <p><strong>Goal:</strong> Design domain adaptation strategies.</p>
</div>

---

### 💡 **Key Observations from Lab Profiles**

#### 🏆 **Dominant Labs:**
- **CalMS21 family** (474 videos, 56% of data) - Uses MARS tracking
- **SparklingTapir** (54 videos) - DeepLabCut, variable FPS (30-120)
- **JovialSwallow** (52 videos) - SLEAP, low FPS (10fps)

#### 🎯 **Critical Variations:**

| Feature | Range | Impact on Model |
|---------|-------|-----------------|
| **FPS** | 10-120 fps | ⚠️ Must normalize by time, not frames! |
| **Body Parts** | 4-18 parts | ⚠️ Need fallback logic for missing parts |
| **Arena Size** | 15-120 cm | ⚠️ Always normalize coordinates |
| **Tracking Method** | MARS, DeepLabCut, SLEAP | ⚠️ Different noise profiles |

#### 🚨 **Watch Out For:**
- `split rectangluar` typo in PleasantMeerkat (should be "rectangular")
- AdaptableSnail has 18 body parts (most detailed tracking)
- BoisterousParrot has 44 hours from just 8 videos (very long recordings!)

---

In [ ]:
# Comprehensive lab profiling
lab_profiles = []

for lab_id in annotated_df['lab_id'].unique():
    lab_data = annotated_df[annotated_df['lab_id'] == lab_id]
    
    # Basic stats
    num_videos = len(lab_data)
    total_duration = lab_data['video_duration_sec'].sum()
    
    # Tracking info
    tracking_methods = lab_data['tracking_method'].unique()
    body_parts_sets = lab_data['body_parts_tracked'].unique()
    
    # Parse body parts
    all_body_parts = set()
    for bp_str in body_parts_sets:
        try:
            parts = json.loads(bp_str)
            all_body_parts.update(parts)
        except:
            pass
    
    # Arena info
    arena_shapes = lab_data['arena_shape'].unique()
    avg_arena_size = lab_data['arena_width_cm'].mean()
    
    # Behavior info
    all_behaviors = set()
    for behaviors_str in lab_data['behaviors_labeled'].dropna():
        try:
            behaviors = json.loads(behaviors_str)
            for b in behaviors:
                # Parse "mouse1,mouse2,action" format
                parts = b.split(',')
                if len(parts) >= 3:
                    all_behaviors.add(parts[2])
        except:
            pass
    
    # FPS
    fps_values = lab_data['frames_per_second'].unique()
    avg_fps = lab_data['frames_per_second'].mean()
    
    lab_profiles.append({
        'lab_id': lab_id,
        'num_videos': num_videos,
        'total_hours': total_duration / 3600,
        'tracking_methods': ', '.join(tracking_methods),
        'num_body_parts': len(all_body_parts),
        'body_parts': ', '.join(sorted(list(all_body_parts))[:5]) + '...' if len(all_body_parts) > 5 else ', '.join(sorted(list(all_body_parts))),
        'arena_shapes': ', '.join(arena_shapes),
        'avg_arena_cm': avg_arena_size,
        'num_behaviors': len(all_behaviors),
        'fps_range': f"{fps_values.min():.0f}-{fps_values.max():.0f}",
        'avg_fps': avg_fps
    })

lab_profiles_df = pd.DataFrame(lab_profiles).sort_values('num_videos', ascending=False)

print("🏢 LABORATORY PROFILES")
print("="*100)
print(lab_profiles_df.to_string(index=False))

---

### 💡 **Understanding Body Part Availability**

#### 🔍 **Why Some Parts Show 0/10 Labs?**

These parts **exist** in the full dataset, but not in the **top 10 labs by video count**. 

**Where they appear:**
- 🔬 **AdaptableSnail** (18 parts) - Has all `headpiece_*` variants
- 🔬 **GroovyShrew** (4 parts) - Has `head` 
- 🔬 **UppityFerret** (14 parts) - Has `spine_*` and `tail_middle_*`

---

### 📊 **Modeling Strategy by Availability:**

| Type | Body Parts | Labs | Strategy |
|------|-----------|------|----------|
| **Universal** | `ear_left`, `ear_right`, `nose`, `tail_base` | 10/10 | ✅ Use directly |
| **Common** | `hip_left`, `hip_right`, `neck` | 6-7/10 | ✅ Use with fallback |
| **Occasional** | `body_center`, `lateral_left/right` | 3/10 | ⚠️ Fallback required |
| **Rare** | `headpiece_*`, `spine_*`, `head` | 0-1/10 | ❌ Don't rely on |

---

### 🎯 **Implementation Pattern:**
```python
def get_body_center(tracking_df):
    """Get body center with cross-lab fallbacks"""
    
    # Priority order: most → least reliable
    priority = ['body_center', 'nose', 'neck', 'ear_left']
    
    for part in priority:
        data = tracking_df[tracking_df['bodypart'] == part]
        if len(data) > 10:  # Min threshold
            return data
    
    # Last resort: use ANY available part
    available = tracking_df['bodypart'].unique()
    if len(available) > 0:
        return tracking_df[tracking_df['bodypart'] == available[0]]
    
    return None

In [ ]:
# Analyze body part tracking across labs
bodypart_by_lab = defaultdict(lambda: defaultdict(int))

for idx, row in annotated_df.iterrows():
    lab_id = row['lab_id']
    try:
        body_parts = json.loads(row['body_parts_tracked'])
        for part in body_parts:
            bodypart_by_lab[lab_id][part] += 1
    except:
        pass

# Create body part matrix
all_body_parts_list = set()
for lab_parts in bodypart_by_lab.values():
    all_body_parts_list.update(lab_parts.keys())
all_body_parts_list = sorted(list(all_body_parts_list))

top_labs = lab_profiles_df.head(10)['lab_id'].tolist()

# Build matrix
bp_matrix = np.zeros((len(top_labs), len(all_body_parts_list)))

for i, lab in enumerate(top_labs):
    for j, part in enumerate(all_body_parts_list):
        bp_matrix[i, j] = bodypart_by_lab[lab].get(part, 0)

# Normalize by number of videos
for i, lab in enumerate(top_labs):
    num_videos = lab_profiles_df[lab_profiles_df['lab_id'] == lab]['num_videos'].iloc[0]
    if num_videos > 0:
        bp_matrix[i, :] = (bp_matrix[i, :] > 0).astype(float)  # Binary: available or not

# Plot
fig, ax = plt.subplots(figsize=(16, 8))

im = ax.imshow(bp_matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=1)

ax.set_xticks(range(len(all_body_parts_list)))
ax.set_yticks(range(len(top_labs)))
ax.set_xticklabels(all_body_parts_list, rotation=45, ha='right', fontsize=10)
ax.set_yticklabels([lab[:25] for lab in top_labs], fontsize=10)

ax.set_xlabel('Body Part', fontsize=12, fontweight='bold')
ax.set_ylabel('Laboratory', fontsize=12, fontweight='bold')
ax.set_title('🎯 Body Part Tracking Availability Across Labs', fontsize=14, fontweight='bold', pad=20)

# Add grid
for i in range(len(top_labs) + 1):
    ax.axhline(i - 0.5, color='gray', linewidth=0.5)
for j in range(len(all_body_parts_list) + 1):
    ax.axvline(j - 0.5, color='gray', linewidth=0.5)

# Colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Available', fontsize=11, fontweight='bold')
cbar.set_ticks([0, 1])
cbar.set_ticklabels(['No', 'Yes'])

plt.tight_layout()
plt.show()

# Summary
print("\n📊 Body Part Tracking Summary:")
print("="*70)
for part in all_body_parts_list:
    num_labs = sum(1 for lab in top_labs if bodypart_by_lab[lab].get(part, 0) > 0)
    print(f"{part:25s} │ Available in {num_labs}/{len(top_labs)} labs")

In [ ]:
# Analyze which behaviors appear in which labs
behavior_by_lab = defaultdict(set)

for idx, row in annotated_df.iterrows():
    lab_id = row['lab_id']
    try:
        behaviors_str = row['behaviors_labeled']
        if pd.notna(behaviors_str):
            behaviors = json.loads(behaviors_str)
            for b in behaviors:
                parts = b.split(',')
                if len(parts) >= 3:
                    action = parts[2]
                    behavior_by_lab[lab_id].add(action)
    except:
        pass

# Get top behaviors and labs
top_behaviors_list = behavior_df.head(20)['behavior'].tolist()
top_labs_list = lab_profiles_df.head(10)['lab_id'].tolist()

# Build matrix
behavior_lab_matrix = np.zeros((len(top_behaviors_list), len(top_labs_list)))

for i, behavior in enumerate(top_behaviors_list):
    for j, lab in enumerate(top_labs_list):
        if behavior in behavior_by_lab[lab]:
            behavior_lab_matrix[i, j] = 1

# Plot
fig, ax = plt.subplots(figsize=(14, 10))

im = ax.imshow(behavior_lab_matrix, cmap='Blues', aspect='auto', vmin=0, vmax=1)

ax.set_xticks(range(len(top_labs_list)))
ax.set_yticks(range(len(top_behaviors_list)))
ax.set_xticklabels([lab[:20] for lab in top_labs_list], rotation=45, ha='right', fontsize=10)
ax.set_yticklabels(top_behaviors_list, fontsize=10)

ax.set_xlabel('Laboratory', fontsize=12, fontweight='bold')
ax.set_ylabel('Behavior', fontsize=12, fontweight='bold')
ax.set_title('🎭 Behavior Annotation Coverage Across Labs', fontsize=14, fontweight='bold', pad=20)

# Add grid
for i in range(len(top_behaviors_list) + 1):
    ax.axhline(i - 0.5, color='white', linewidth=1)
for j in range(len(top_labs_list) + 1):
    ax.axvline(j - 0.5, color='white', linewidth=1)

# Colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Annotated', fontsize=11, fontweight='bold')
cbar.set_ticks([0, 1])
cbar.set_ticklabels(['No', 'Yes'])

plt.tight_layout()
plt.show()

# Identify universal vs lab-specific behaviors
print("\n🌍 Behavior Generalization Analysis:")
print("="*70)

universal_behaviors = []
lab_specific_behaviors = []

for behavior in top_behaviors_list:
    num_labs = sum(1 for lab in top_labs_list if behavior in behavior_by_lab[lab])
    coverage = num_labs / len(top_labs_list)
    
    if coverage >= 0.5:
        universal_behaviors.append((behavior, num_labs))
    else:
        lab_specific_behaviors.append((behavior, num_labs))

print("\n✅ Universal Behaviors (>50% lab coverage):")
for beh, count in universal_behaviors:
    print(f"  • {beh:20s} → {count}/{len(top_labs_list)} labs")

print("\n⚠️ Lab-Specific Behaviors (<50% lab coverage):")
for beh, count in lab_specific_behaviors[:10]:
    print(f"  • {beh:20s} → {count}/{len(top_labs_list)} labs")

---
# 🎯 Section 6: Actionable Modeling Recommendations

<div style="background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); padding: 25px; border-radius: 10px; color: white;">
    <h2 style="margin: 0;">🏆 Competition Strategy Summary</h2>
</div>

## 📊 Data Strategy

### 1️⃣ **Train/Validation Split Design**

**Recommended approach:**
- Use stratified GroupKFold by lab_id (5 folds)
- Ensure each fold has diverse labs for robust generalization
- Reserve 1-2 labs completely for final validation

### 2️⃣ **Class Imbalance Handling**

| Strategy | When to Use |
|----------|-------------|
| **Focal Loss** | For behaviors with <100 examples |
| **Class Weights** | All behaviors, inverse frequency weighting |
| **Oversampling** | Rare behaviors (<50 examples) |
| **Mixup/Cutmix** | Data augmentation for temporal sequences |

### 3️⃣ **Domain Adaptation**

**Must-have techniques:**
- ✅ Coordinate normalization by pix_per_cm_approx
- ✅ FPS-agnostic features (use seconds, not frames)
- ✅ Body-part-agnostic fallbacks (nose → body_center → head)
- ✅ Lab-ID as auxiliary input (domain classifier)

---

## 🔧 Feature Engineering Priorities

### **Tier 1: Essential Features** (Implement First)
1. **Velocity & Acceleration** - Rolling windows: 5, 10, 30 frames
2. **Inter-mouse distances** - For all mouse pairs
3. **Arena-relative position** - Distance to center, walls, corners
4. **Angular features** - Body orientation, turning rate
5. **Temporal aggregations** - Mean, std, min, max over multiple windows

### **Tier 2: Advanced Features** (Add for improvement)
6. **Behavior history** - Previous behavior encoding (1-hot or embedding)
7. **Relative velocities** - Between mouse pairs
8. **Trajectory curvature** - Second derivatives
9. **Pose configuration** - Body part relative positions
10. **Interaction features** - Approach rate, alignment, following

### **Tier 3: Experimental** (Test carefully)
11. **Fourier features** - Frequency domain representations
12. **Graph neural features** - Mouse-mouse interaction graphs
13. **Attention features** - Self-attention over body parts

---

## 🏗️ Model Architecture Recommendations

### **Baseline Models** (Start here)

**1. LightGBM with hand-crafted features**
- Fast iteration, interpretable
- Good for understanding feature importance

**2. 1D CNN on raw pose sequences**
- Learns temporal patterns automatically
- Handles variable-length sequences

### **Advanced Models** (For leaderboard push)

**3. Temporal Convolutional Network (TCN)**
- Better than LSTM for long sequences
- Parallel computation advantage

**4. Transformer-based**
- Multi-head attention over time
- Positional encoding for temporal order

**5. Ensemble Strategy**
- LightGBM + TCN + Transformer
- Lab-specific models + global model

---

## ⚠️ Critical Pitfalls to Avoid

<div style="background-color: #ffebee; padding: 15px; border-left: 5px solid #f44336; border-radius: 5px;">
    <h3>🚫 Don't Do These!</h3>
    <ul>
        <li><strong>Leakage:</strong> Using future frames in rolling features (use center=False)</li>
        <li><strong>Overfitting to CalMS21:</strong> This lab dominates data - validate on other labs!</li>
        <li><strong>Ignoring FPS:</strong> Different videos have different frame rates</li>
        <li><strong>Hard-coding body parts:</strong> Not all labs track same parts</li>
        <li><strong>Static thresholds:</strong> Adaptive thresholds per behavior perform better</li>
    </ul>
</div>

---

## 📈 Evaluation Strategy

### **Validation Metrics**
- **Primary:** Per-lab averaged F1 (matches competition metric)
- **Secondary:** Confusion matrix for behavior-level analysis
- **Tertiary:** Boundary precision (±5 frame tolerance)

### **Error Analysis**

Focus debugging on:
1. **False Negatives** for rare behaviors → Increase sensitivity
2. **False Positives** for common behaviors → Increase specificity  
3. **Boundary errors** → Post-processing smoothing
4. **Cross-lab failures** → Domain adaptation

---

## 🎬 Next Steps

<div style="background-color: #e8f5e9; padding: 15px; border-left: 5px solid #4caf50; border-radius: 5px;">
    <h3>✅ Implementation Checklist</h3>
    <ol>
        <li>✏️ Build baseline with hand-crafted features + LightGBM</li>
        <li>📊 Implement proper cross-validation (GroupKFold by lab)</li>
        <li>🎯 Add class weights for imbalance</li>
        <li>🔧 Engineer Tier 1 features</li>
        <li>🧪 Experiment with temporal models (TCN/Transformer)</li>
        <li>🏆 Ensemble multiple models</li>
        <li>⚡ Optimize post-processing (smoothing, merging short events)</li>
    </ol>
</div>

---

<div style="text-align: center; padding: 20px; background-color: #f5f5f5; border-radius: 10px;">
    <h3>🐭 Happy Modeling! 🐭</h3>
    <p><em>"In mouse behavior analysis, the devil is in the temporal details."</em></p>
</div>

---

### 🙏 Thank you for reading!

**Good luck in the competition!** 🚀🐭

*If this notebook helped you, please upvote!* ⭐